####  Data Transformation Process
##### Model the data for the expected queries

##### 1. Dependencies

In [1]:
import os
import pandas as pd
import numpy as np

##### 2. Load the Data generated by the Web Application (JSON)

In [3]:
web_data ='/Users/William/Workspace/Devoteam/data/from_branches/webapp_data.json'


##### 3. Data exploration and transformation

In [5]:
df = pd.read_json(web_data)
df.tail()

,client,district,id_branch,infection_confirmed,latitude,longitude,patient,time_stamp_date,time_stamp_hour
995,True,6,38,False,59.927607,32.296821,"{'age': 22, 'gender': 'Female', 'travel': True...",3/6/2015,7:15 PM
996,True,3,31,False,10.256885,-67.168595,"{'age': 6, 'gender': 'Female', 'travel': False...",2/17/2015,1:19 AM
997,False,2,41,True,64.162979,24.306788,"{'age': 52, 'gender': 'Male', 'travel': False,...",5/19/2015,8:33 AM
998,False,1,46,True,-8.219061,128.941023,"{'age': 23, 'gender': 'Female', 'travel': True...",3/19/2015,4:59 AM
999,True,4,6,False,49.137643,9.177606,"{'age': 73, 'gender': 'Male', 'travel': True, ...",5/20/2015,5:10 AM


##### 4. Tables building -- What we load in Cassandra DB

##### 4. 1. Query: Please tell me the confirmed Zika Virus cases sorted primarly by Age

In [13]:
df_confirmed = df[df['infection_confirmed'] == True]
print('Based on the Data, we have : ', len(df_confirmed), 'Confirmed')

Based on the Data, we have :  510 Confirmed


##### 4.2. We want only Patient Information for this query 

In [14]:
df_confirmed = df_confirmed['patient']

In [15]:
df_c = pd.DataFrame(df_confirmed)
df_c.reset_index(drop = True)
df_c.head()

,patient
0,"{'age': 76, 'gender': 'Female', 'travel': True..."
1,"{'age': 25, 'gender': 'Female', 'travel': True..."
3,"{'age': 42, 'gender': 'Female', 'travel': True..."
4,"{'age': 11, 'gender': 'Female', 'travel': True..."
5,"{'age': 4, 'gender': 'Male', 'travel': False, ..."


In [16]:
df_c = df_c.reset_index(drop=True)
df_c.head()

,patient
0,"{'age': 76, 'gender': 'Female', 'travel': True..."
1,"{'age': 25, 'gender': 'Female', 'travel': True..."
2,"{'age': 42, 'gender': 'Female', 'travel': True..."
3,"{'age': 11, 'gender': 'Female', 'travel': True..."
4,"{'age': 4, 'gender': 'Male', 'travel': False, ..."


#### 4.3 From the column Patient, I build a new data structure with the fields I need for my query, where Age is a primary Key 
##### One patient Example:

In [17]:
data = df_c['patient'][500]
data

{'age': 25,
 'gender': 'Female',
 'travel': True,
 'fever': True,
 'id': 981,
 'pregnant': True}

In [18]:
df_table_ages = pd.DataFrame()
for iterator in range(0, len(df_c)):
    df_table_ages = df_table_ages.append(pd.DataFrame(df_c['patient'][iterator], index = [0])) 

##### 4.4 Table with Age as Primary Key to be Ingested in the Database

In [25]:
df_ages_f = df_table_ages.loc[:, ['id', 'age', 'travel', 'fever']]
df_ages_f.to_csv("outAgeDataset.csv", index=False)

   id  age  travel  fever
0   1   76    True   True
0   2   25    True  False
0   4   42    True  False
0   5   11    True   True
0   6    4   False   True


##### Exported Data as CSV for potentially be used by other Visualization tools

In [27]:
df_ages_f.head()

,id,age,travel,fever
0,1,76,True,True
0,2,25,True,False
0,4,42,True,False
0,5,11,True,True
0,6,4,False,True
